In [ ]:
%load_ext autoreload
%autoreload 2

# ENLEVER LES POI?TS GEOGRAPHIQUES DUPLIQUES

In [ ]:
from pathlib import Path
import gstlearn as gl
import gstlearn.plot as gp
from bramm_data_analysis.spatial import RegularGrid

# Data

### Grid

In [ ]:
step = 0.1
grid = RegularGrid.from_boundary_path(Path("../data/metropole.json"))
# france_grid = grid.retrieve_insiders(step)
# fr_db = gl.Db_fromPanda(france_grid)

### Loading

In [ ]:
from bramm_data_analysis import loaders

# Moss
moss_data_path = Path("../data/Mines_2024.xlsx")
df_moss = loaders.from_moss_csv(moss_data_path).retrieve_df()

In [ ]:
from sklearn.preprocessing import StandardScaler

x1 = "longitude"
x2 = "latitude"
z1 = "copper"
z2 = "aluminium"

df = df_moss.filter([x1, x2, z1]).drop_duplicates(subset=[x1, x2])

# df[z1] = StandardScaler().fit_transform(df[z1].to_frame().resize((-1,1)))

observations = gl.Db_fromPanda(df)
observations.setLocators([x1, x2], gl.ELoc.X)
observations.setLocator(z1, gl.ELoc.Z)

In [ ]:
extrems = observations.getExtremas()

In [ ]:
extrems

In [ ]:
db_grid = gl.DbGrid.create(
    x0=[extrems[0, 0], extrems[1, 0]],
    dx=[step, step],
    nx=[
        int((extrems[0, 1] - extrems[0, 0]) / step),
        int((extrems[1, 1] - extrems[1, 0]) / step),
    ],
)

In [ ]:
varioParamMulti = gl.VarioParam.createMultiple(
    ndir=2, npas=10, dpas=0.5, toldis=0.2
)
vario2dir = gl.Vario(varioParamMulti)
err = vario2dir.compute(observations)
ax = gp.varmod(vario2dir)

In [ ]:
fitmod = gl.Model()
types = [gl.ECov.NUGGET, gl.ECov.EXPONENTIAL, gl.ECov.GAUSSIAN]
err = fitmod.fit(vario2dir, types=types)
ax = gp.varmod(vario2dir, fitmod)

In [ ]:
observations.setLocator(z1, gl.ELoc.Z)
observations

In [ ]:
uniqueNeigh = gl.NeighUnique.create()

In [ ]:
err = gl.kriging(
    dbin=observations,
    dbout=db_grid,
    model=fitmod,
    neigh=uniqueNeigh,
    flag_est=True,
    flag_std=True,
    flag_varz=False,
    namconv=gl.NamingConvention("SK"),
)

In [ ]:
db_grid["SK.copper.estim"]

In [ ]:
db_grid.plot("SK.copper.estim")